# **BDTs at work: the $\mathrm{D}^{+}_{s}$ analysis**

The goal of this tutorial is to train and test a multiclass classification algorithm employing machine learning (ML) techniques to perform a typical ALICE study in the HF sector. This tutorial focuses on the measurement of the invariant-mass distribution of the $\mathrm{D}^{+}_{s}$ meson. The $\mathrm{D}^{+}_{s}$ meson is reconstructed in its hadronic decay channel $\mathrm{D}^{+}_{s} \rightarrow \mathrm{\phi} + \pi^+ + K^+ + K^- + \pi^-$. The $\mathrm{D}^{+}_{s}$ mesons are typically categorised as either $prompt$ (i.e. origintaing from charm quark hadornisation) or $non-prompt$ (i.e. origintaing from beauty-hadron decay).

The ML algorithm is going to classify the $\mathrm{D}^{+}_{s}$ meson candidates as: prompt, non-prompt, or background.
Hence, we need to train it with examples from all the classes. For the signal we will use MC simulations, while for the background we will use the data collected in pp collisions by ALICE in Run 3.
In particular:
- MC production: LHC22b1b (charm-enriched), LHC22b1a (beauty-enriched)
- Data: LHC22o_pass4

<img src="img/DsDecaySketch.png" 
     align="center" 
     width="900" />

### **File download**

Let's firts download all the samples we need. In Run 3 we can save the tables produced by O2Physics analysis task as dervied data in the form of Trees. This can be done directly on hyperloop or via TreeCreators (see LINK).
To speed up this part, the AO2D from hyperloop have been filtered and dumped in .parquet.gzip files (thanks fchinu!)

!curl -L ht\mathrm{\mathrm{\mathrm{\mathrm{TP}}}}s://github.com/hipe4ml/hipe4ml_tests/blob/master/PID7TeV/pi_fromV0_data_filtered.parquet.gzip?raw=true --output promptDs.parquet.gzip

!curl -L https://github.com/hipe4ml/hipe4ml_tests/blob/master/PID7TeV/kaons_fromkinks_data_filtered.parquet.gzip?raw=true --output nonpromptDs.parquet.gzip

!curl -L https://github.com/hipe4ml/hipe4ml_tests/blob/master/PID7TeV/protons_fromL_data_filtered.parquet.gzip?raw=true --output Data.parquet.gzip

## Boosted Decision Tree in a nutshell

Supervised learning, a well-established subfield within the realm of Machine Learning, holds significant relevance in High Energy Physics (HEP). In this context, supervised learning techniques are instrumental to slove classification problems. The process begins with a training set, comprising a set of examples. Each element of this training set is assigned a label indicating its class. In the physics case under study: prompt, non-prompt, or background. In the training the class information is known in advance. The training process focuses on optimising the internal parameters of the ML algorithm to maximise the separation power between the classes. The primary objective of the training is to impart the model with common data patterns that can be exploited for accurately classify an independent sample, the data collected by ALICE for instance. The supervised model associates to each candidate an \emph{output score}, which depends on the properties of the candidate itself, referred to as $features$. The score is linked to the probability of the candidate belonging to one of the different classes.

In this tutorial, $Boosted\;Decision\;Trees$ (BDTs) are employed to tag prompt and non-prompt $\mathrm{D}^{+}_{s}$ candidates. The building block of every BDT model is the $Decision\;Tree$ algorithm (DT). A decision tree is a binary structure that resembles a flowchart. In this structure, internal nodes represent features or candidates, branches indicate decision rules, and leaf nodes represent the final outcomes. The DT operates by employing a sequence of simple binary tests, each corresponding to a branch in the tree, to classify a data point based on its features. These tests involve applying a linear threshold to one of the features, aiding the model in predicting the class to which every candidate belongs. Training a decision tree involves an automated process that recursively constructs the tree using the training set. However, a key limitation of decision trees is their susceptibility to $overfitting$. This means that, if a decision tree is deep enough (where depth is defined as the length of the longest path from the root to a leaf), it can perfectly classify the training set but may struggle to generalise to new data. Overfitting occurs when the model essentially "memorises" the training set rather than learning more general patterns in data. To mitigate this issue, BDT algorithms are employed, which combine multiple shallow trees, each using a subset of features. In the boosting procedure, decision trees are constructed sequentially, with an emphasis on compensating for misclassified candidates from the previous trees. The resulting model, the BDT, maintains strong performance on both the training and test sets.

### **Required python packages**

First we import standard python packages

In [ ]:
### standard sci-py libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

Then we import the _hipe4ml_ package: https://github.com/hipe4ml/hipe4ml


<img src="img/hipe4ml.png" 
     align="center" 
     width="300" />

This is a package developed in ALICE containing useful methods and classes for dealing with ML analyses.
Two main classes are implemented:
- _TreeHandler_, wrapping uproot and pandas methods: allows for conversion and handling of the training samples
- _ModelHandler_, a common interface for many ML methods

In [ ]:
from hipe4ml.model_handler import ModelHandler
from hipe4ml.tree_handler import TreeHandler
from hipe4ml import analysis_utils
from hipe4ml import plot_utils

### **Preparing the training set and the test set**

All the data sets are in _parquet_ data format and are all obtained from real data taken by the ALICE Collaboration as described in the introduction.
We now open the parquet with the _TreeHandler_ class to facilitate the handeling of the data for ML purposes.  

In [ ]:
hdl_mc_prompt = TreeHandler("AnalysisResults-mc_charm_reduced.root", "XiOmegaTree")
hdl_mc_nonprompt = TreeHandler("AnalysisResults-mc_beauty_reduced.root", "XiOmegaTree")
hdl_data = TreeHandler("AnalysisResults-data_reduced.root", "XiOmegaTree")

## we only consider the sidebeand region in the data
hdl_data.apply_preselections("mass < 1.82 and mass > 2.02", inplace=True)

hdl_all = [hdl_mc_prompt, hdl_mc_nonprompt, hdl_data]

A _total set_ is built by merging the individual sets. A label is assigned to each candidate of the total set: 0 for prompt Ds, 1 for non-prompt Ds, and 2 for background.
The total set is split into a _train set_ and a _test set_, used for training and testing the algorithm, respectively.
The fraction of the total set used as test set is defined by the _test_size_ parameter of the function [_sklearn.model_selection.train_test_split_](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) and is tipically around 0.2. 

In [ ]:
train_test_data = train_test_generator(hdl_all, [0, 1, 2], test_size=0.2, random_state=42)

The distributions of the features for signal and background candidates are plotted using the function _plot\_utils_._plot\_distr_ of hipe4ml. Similarly, the correlation matrix for the features is plotted with the function _plt\_utils_._plot\_corr_.

In [ ]:
vars_to_draw = [] ##just a selection of all the  possible variables
leg_labels = ["prompt", "nonprompt", "bakground"]

plot_utils.plot_distr(hdl_all, vars_to_draw, bins=100, labels=leg_labels, log=True, density=True, figsize=(12, 7), alpha=0.3, grid=False)
plt.subplots_adjust(left=0.06, bottom=0.06, right=0.99, top=0.96, hspace=0.55, wspace=0.55)
plot_utils.plot_corr(hdl_all, vars_to_draw, leg_labels)

plt.show()

### **The model**

For this tutorial, [_XGBoost_](https://xgboost.readthedocs.io/en/latest/) is used as classification algorithm. XGBoost is an implementation of gradient boosted decision trees, designed to be highly efficient, flexible and portable.

The _hipe4ml_ package deals with the model through the _ModelHandler_ module. The model-handler is used to define the features used in the training process and to set the _hyperparameters_ of the model, such as the number of estimators, the maximum depth of the trees and the learning rate.

Let's initialise the algorithm with the features we want to include in the training.


In [ ]:
features_for_train = []

model_clf = xgb.XGBClassifier()
model_hdl = ModelHandler(model_clf, features_for_train)

### **Optimisation of hyperparameters with Optuna**
The _hyperparameters_ consist of the internal parameters of the BDT algorithm which are not optimised within the training phase. Few of them are listed here, for a complete description see : https://xgboost.readthedocs.io/en/stable/parameter.html
- _n_estimators_: Number of trees in the BDT
- _max_depth_: Maximum depth of a tree
- _eta_: learning rate of the algorithm. It controls the step size of the gradient descent algorithm

The optimisation of the hyperparameters is a key step to obtain the best performance from the algorithm. In _hipe4ml_ the Optuna package is employed for the optimisation through the method _ModelHandler.optimize\_params\_optuna_. (<https://github.com/optuna/optuna>)

The difference between Optuna and other packages for hyperparametrs optimisation based on grid search or random search is that Optuna takes into account the past hyperparameter configuration evaluations when choosing the configuration to evaluate next.

A set of hyperparameters should be tested on different samples to avoid overfitting problems. Since the number of events is limited, an approach called _k-flod cross validation_ is used. It has been proved that the cross validation removes the dependence of the model on the data sample. 

In the cross validation procedure, the original sample is divided in _k_ parts called _folds_ (in this case 5 folds are used). For each set of hyperparameters, _n-1_ folds are used for the optimisation and the remaining one as test. This operation is repeated after permuting the folds used for optimisation and for testing and the final result is the mean value of all the permutations.

The ModelHandler automatically updates the hyperparameters after their optimisation.

To spare time, in this tutorial the number of hyperparameters and the number of configuration is limited.
The full operation has been conducted locally and the resulting values are reported below

In [ ]:
hyper_pars_ranges = {"n_estimators": (200, 1000)}
model_hdl.optimize_params_optuna(train_test_data, hyper_pars_ranges, cross_val_scoring="roc_auc_ovo", timeout=60, n_jobs=1, n_trials=5,direction="maximize")
hyper_pars_opt = {"n_estimators": 1000}

### **Training and testing the model**
The model training is performed with the method _train\_test\_model_ of the _ModelHandler_ module. It requires as argument a list containing:
1. the training set;
2. the real class label for the training set;
3. the test set;
4. the real class label for the test set;
5. the multiclass option.

The multiclass option can be set to _ovo_ (i.e._OneVsOne_) or _ovr_ (i.e._OneVeRest_). In the first configuration, for each possible couple of classes a BDT algoirithm is trained considering the first class as signal and the second one as background. In the second configuration, for each class a BDT algoirithm is trained considering that class as signal and the others as background. In both cases, the final score associated with a candidate is computes as an average of the different BDT scores.

The predictions for the training and the test sets are obtained with the _predict_ method.

In [ ]:
model_hdl.train_test_model(train_test_data, multi_class_opt="ovo")

y_pred_train = model_hdl.predict(train_test_data[0], False)
y_pred_test = model_hdl.predict(train_test_data[2], False)

The results of the training process can be observed by plotting the distributions of the BDT scores for the training and the test sets. The output consists of a score related to the probability to belong to each of the classes used for the training. This operation is performed with the method _plot\_utils_._plot\_output\_train\_test_.

The distributions of the model scores obtained from the test set are in good agreement with those obtained from the training set. This is a sign that tha model has been trained properly.

A disagreement between the distributions obtained from the training set and the datasets would reflect an overfitting by the classification algorithm: the classifier has learnt some characteristics that are peculiar of the training set but that are not true for a general sample.

In [ ]:
plt.rcParams["figure.figsize"] = (10, 7)

ml_out_fig = plot_utils.plot_output_train_test(model_hdl, train_test_data, 100, 
                                               False, leg_labels, True, density=True)

### **Receiver Operating Curve**
The quality of the algorithm can be also studied with the _ROC curve_. The ROC curve is defined as a function of the _True Positive Rate_ (TPR) against the _False Negative Rate_ (FPR) as a function of a threshold on the BDT score. TPR and FPR are defined as:

$\mathrm{TPR}=\frac{\sum \mathrm{TP}}{\sum \mathrm{TP} + \sum \mathrm{FN}} \hspace{2cm} \mathrm{FPR}=\frac{\sum \mathrm{FP}}{\sum \mathrm{FP} + \sum \mathrm{TN}} $

In the case of multiclassifiaction problems, the ROC curve is not natively defined. Hence, we can either employ the OneVsOne or the OneVs Rest approach. In this tutorial, we adopte the OneVsOne. Therefore, the TPR and the FPR represent the selection efficeincy for the first class, considered as signal, and the rejection
efficiency of the second class, considered as background, for each couple of classes, respectively.

The most common way employed to evaluate the performance of a BDT is to compute the _Area Under the Curve_ ROC, called AUC:
- a perfect classifier will have a ROC AUC equal to 1;
- a random classifier will have a ROC AUC equal to 0.5.

A good model classifier is characterised by a large area under the ROC curve (_ROC AUC_).

The ROC curve can be plotted with the method _plot\_utils_._plot\_roc\_train\_test_. 

In [ ]:
roc_train_test_fig = plot_utils.plot_roc_train_test(train_test_data[3], y_pred_test, None, leg_labels, multi_class_opt="ovo")

Evaluating the ROC AUC in training and test a slight difference is found. The higher ROC AUC in training compared to test is a systematic behaviour due to the small presence of overfitting, which indicates a successful training.

### **Feature Importance**

It is crucial to assess which feature is more important for the BDT to assign its score. This step is twofold. On one hand, understanding which features are the most important to the BDT can help us to interpret the outcome of the algorithm. On the other hand, the feature importance can be used as a cross-check to ensure that the algorithm predilection for
a specific feature is matched by the physics of the problem under study. For instance in the case of D mesons, the most effective features are expected to be the ones associated with the decay length.

The feature importance implemented in the _SHAP library_ (https://github.com/slundberg/shap) is used in this hands-on session. In the context of machine learning, the _Shapley value_ is used to evaluate the contribution of each feature to the model output, and it is calculated by averaging the marginal contributions of each feature to the model output. The marginal contribution of a feature is the difference in the model output when the feature is present or absent. The variables that are more important for the model are those that have a higher marginal contribution, and Shapley values consequently.

The SHAP feature importance can be plotted witht the method _plot\_utils_._plot\_feat\_imp_

In [ ]:
plot_utils.plot_feature_imp(train_test_data[2], train_test_data[3], model_hdl) 

### **Visualise the results**

